# Лабораторная 3 - Знакомство со сверточными нейронными сетями; обучение нейронных сетей

В этой лабораторной вы познакомитесь что такое сверточные нейронные сети (Convolutional Neural Networks, CNN), как их применять для задачи классификации изображений и как правильно организовать процесс обучения нейронных сетей.

Сыерточные нейронные сети были разработаны французским ученым Яном Лекуном (Yann LeCun) в конце 80-х годов. Они сразу показали свою высокую эффективность при обработке изображений. Это происходит из-за того, что сеть не только учится воспринимать значения пикселей, но и учится распознавать отношения между рядом стоящими пикселями.

Простейшая многослойная сверточная сеть состоит из трех типов слоев: сверточных слоев, слоев пулинга и полносвязных слоев (с которыми вы познакомились в предыдущей лабораторной).

То, как работает слой свертки показано на следующей анимации:

![работа сверточного слоя](img/ConvLayer.gif)

Слева на этой анимации показана матрица входных данных. Для простоты, можно домуть о ней как об одноканальном черно-белом изображении, состоящем из пикселей. 
Красным квадратом показано окно свертки. В данном случае, оно имеет размер 3 x 3 пикселя. Каждое значение из этого окно домножается на определенный вес (это те самые веса, которые будут корректироваться в ходе обучения сети). Матрица весов представлена в середине рисунка. Фильтр скользит по исходному изображению с определенным шагом.
Справа показана матрица, получающаяся в результате применения фильтра свертки.

Предположим, что в данный момент времени, центральный пиксель фильтра находится в координате (1, 1). Посчитаем, какое значение фильтра будет в этом случае:
$$(7 * 0) + (6 * (-1)) + (5 * 0)+$$
$$(6 * (-1)) + (4 * 5) + (3 * (-1))+$$
$$(5 * 0) + (3 * -1) + (2 * 0)=2$$

У сверточного слоя таких фильтров может быть неколько. Они работают параллельно, независимо друг от друга. Количество сверточных фильтров в рамках одного слоя называется количеством каналов исходного тензора (напоминаю, что тензором называется многомерная матрица). 

Помимо количества фильтров, у сверточных слоев есть два дополнительных параметра. Первый из них - padding. Он показывает на сколько пикселей нужно расширить каждую сторону исходной матрицы, когда фильтр дойдет до этой стороны. На анимации сверху padding равняется одному. Второй параметр - strides. Он задает с каким шагом перемещается окно фильтра. В примере выше strides равен 1, потому что фильтр шагает вправо и вниз с шагом 1. Детальную визуализацию того, как работает padding и strides можно увидеть по следующей ссылке [Padding и Strides](https://github.com/vdumoulin/conv_arithmetic/blob/master/README.md)

Теперь перейдем к рассмотрению pooling слоев. Визуализация работы pooling слоя представлена на следующем рисунке:

![MaxPooling](img/Max_pooling.png)

У pooling слоев тоже есть окна, с которыми они предвигаются по изображению. Но вместо того, чтобы брать сумму произведений пикселей на веса, они просто берут максимальное/минимальное/среднее значение пикселей из области окна. Когда берется максимальное значение, pooling называется MaxPooling; минимальное значение - MinPooling; среднее значение - AvgPooling. Как видно из описания, в pooling-слоях нет весов, соответственно, они не являются обучаемыми слоями. 

На рисунке сверху представлен пример работы MaxPooling. Каждая итерация работы окна показана разным цветом. Слева представлено исходная матрица, а справа - получившийся результат. Видно, что в отличие от сверточных слоев при скольжении окна по исходной матрице не происходит наложение одного окна на другое.

Теперь давайте применим полученные знания на практике и построим классификатор изображений, основанный на сверточных нейронных сетях.

Для начала импортируем библиотеку PyTorch и библиотеку torchvision, в которой содержится много полезных функций для работы с изображениями. Для установки библиотеки torchvision необходимо использовать команду `pip install torchvision`

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

Многие фреймворки машинного обучения могут производить вычисления не только на CPU, но и на видеокартах - GPU. Вычисление на GPU значительно повышает скорость вычислений. По этому, если в вашем распоряжении есть видеокарта с поддержкой Nvidia CUDA, то вы можете использовать ее для работы с нейронными сетями. Для того, чтобы определить есть ли в вашем компьютере такая видеокарта или нет, в PyTorch есть функция torch.cuda.is_available( ), которая возвращает True, если GPU имеется и False в противном случае. Чтобы использовать в дальнейшем это устройство, нужн осоздать экземпляр утройства это делается с помощью torch.device( ) в который нужно передать, либо 'cuda', если вы хотите вычислять на GPU, либо 'cpu' - если на центральном процессоре

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("cuda" if torch.cuda.is_available() else "cpu")


cpu


In [3]:
import subprocess
# subprocess.run(["ls"])
proc = subprocess.Popen(["ls"], stdout=subprocess.PIPE, shell = True )
out, err = proc.communicate()
print(out.decode('utf-8'))

Далее подготовим данные для обучения нашей сети. В качестве данных мы будем использовать датасет CIFAR10. Этот датасет состоит из картинок, на каждой из которых изображен какой-то объект. Всего классов таких объектов 10 (самолет, легковая машина, птица, кошка и т.д.). 

Когда все обучающие примеры пройдут через сеть закончится одна эпоха обучения. Таких эпох может быть несколько. После каждой эпохи обучения принято тестировать модель, чтобы проверить то, как она реагирует на новые, еще не увиденные ею данные. По весь набор данных принято разбивать на два подмножества - одно для обучения (обучающая выборка) и одно для тестирования (тестовая выборка). Обычно, в обучающую выборку попадает 70% всех данных. 

Кроме того, чтобы обучение сети проходило более эффективно, принято перемешивать обучающую выборку. Это делается с той целью, чтобы данные, относящиеся к одним и тем же классам не стояли рядом друг с другом. 

Напоминаю, что вам уже известно, что сеть может обрабатывать несколько примеров за один раз. Это называется батчем. В данном случае размер батча равен 8.

In [4]:
# Указываем, какие трансформации нужно производить с исходными данными
# перед тем, как они попадут в сеть.
# Чтобы задать набор трансформаций, нужн овоспользоваться классом
# transforms.Compose( ), в который нужно передать список из классов-трансформаций.
# В данном случае мы используем преобразование исходных данных в тензор PyTorch: transforms.ToTensor( )
# и нормализуем изображение, чтобы каждый пиксель принимал значение из нормального распределения.
# В данном случае, у нас изображение является трехканальным, по этому мы указываем наборы из трех значений.
# Первый набор - среднее значение нормального распределения, а второе значение - его среднеквадратическое отклонение
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Создаем набор данных для обучения. Для этого мы будеем использовать уже имеющийся в библиотеке датасет.
# Параметр root показывает, куда скачивать датасет;
# train - показывает, что эти обучающая выборка;
# download - показывает, что данных у нас нет в директории root и их необходимо скачать
# transform - мы указываем, какие трансформации нужно произвести с каждым изображением.
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
# После того, как данные загружены, мы создаем генератор батчей.
# В него мы передаем наши данные.
# batch_size - размер батча
# shuffle - нужно ли перемешивать данные
# num_workers - сколько ядер процессора использовать для создания генератора
trainloader = torch.utils.data.DataLoader(trainset, batch_size=8,
                                         shuffle=True, num_workers=2)

# Загружаем данные для тестирования сети
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True, transform=transform)
# Создаем генератор батчей для тестового набора данных
testloader = torch.utils.data.DataLoader(trainset, batch_size=8,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog',
          'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


Теперь напишем архитектуру нашего классификатора

In [5]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    # Конструктор нашей сети, в котором описываются все слои, которые будут в ней использоваться
    def __init__(self):
        super(Net, self).__init__()
        # При конструировании архитектур сетей очень важно уделять внимание размерности данных на каждом каждом этапе работы сети.
        # Это позволяет контролировать то, что происходит на каждом слое и избегать несогласованности размерностей
        # Задаем переменную-флаг, чтобы выводить информацию только о первом батче
        self.first_batch = True
        
        # Сверточный слой для обработки двухмерных изображений описываются с помощью слоя Conv2d.
        # В конструктор передаются следующие параметры:
        # первый параметр - количество каналов входных данных (их 3, потому что у нас изображение цветное)
        # второй параметр - количество каналов выходного изображения (т.е. количество фильтров свертки)
        # третий параметр - размер скользящего окна (в этом случае оно имеет размер 5 х 5)
        self.conv1 = nn.Conv2d(3, 6, 5)
        # Для применения MaxPooling-а к двухмерной матрице используется слой MaxPool2d
        # В конструктор класса передаются следующие параметры:
        # первый параметр - размер окна pooling-а (в нашем случае окно 2 х 2)
        # второй параметр - на сколько сдвигать окно на каждой итерации
        # (в большинстве случаев этот параметр совпадает с размерами окна)
        self.pool = nn.MaxPool2d(2, 2)
        # Второй сверточный слой
        # число входных каналов совпадает с числом выходных каналов у предыдущего сверточного слоя
        # число выходных каналов - 16
        # размер окна свертки - 5 x 5
        self.conv2 = nn.Conv2d(6, 16, 5)
        
        # Для проведения классификации изображений используется трехслойная полносвязная сеть
        # Первый слой этой сети имеет размерность входных данных - 400.
        # Это число берется из следующего расчета: 16 каналов после второй свертки * 
        # 5 * 5 (размер матрицы изображения, получившийся после применения сверток и pooling-а).
        # Число нейронов на этом слое - 120.
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        # Второй полносвязный слой
        # Размерность входных данных - 120 (совпадает с количеством нейронов на предыдущем слое)
        # Число нейронов - 84
        self.fc2 = nn.Linear(120, 84)
        # Третий полносвязный слой
        # Размерность входных данных - 84
        # Число нейронов - 10 (важно, чтобы число нейронов на последнем слое классификатора совпадала с числом классов)
        self.fc3 = nn.Linear(84, 10)
    
    # Задаем последовательность взаимодействия слоев сети
    def forward(self, x):
        # Выводим размерность входных данных у первого батча
        # Первая координата - размер батча
        # Вторая координата - количество каналов
        # Третья и четвертая координата - высота и ширина матрицы изображения
        if self.first_batch:
            # (8, 3, 32, 32)
            print('Input data shape: {}'.format(x.shape))
            
        # Применяем первый сверточный слой.
        # Для этого слоя используем функцию активации ReLU.
        x = F.relu(self.conv1(x))
        # Выводим размерность данных после первого сверточного слоя
        if self.first_batch:
            # (8, 6, 28, 28)
            print('Data shape after 1-st conv: {}'.format(x.shape))
        
        # Применяем слой MaxPooling
        x = self.pool(x)
        # Можно заметить, что размер матрицы изображения уменьшился в 2 раза.
        # Это происходит из-за того, что для pooling-а мы использовали окно 2
        if self.first_batch:
            # (8, 6, 14, 14)
            print('Data shape after 1-st pooling: {}'.format(x.shape))
        
        # Применяем второй сверточный слой.
        # Для этого слоя используем функцию активации ReLU.
        x = F.relu(self.conv2(x))
        if self.first_batch:
            # (8, 16, 10, 10)
            print('Data shape after 2-nd conv: {}'.format(x.shape))
        
        # Снова применяем слой MaxPooling. Напоминаю, что в слое MaxPooling-а нет обучаемых параметров.
        # По этому можно применять один и тот же слой несколько раз.
        x = self.pool(x)
        if self.first_batch:
            # (8, 16, 5, 5)
            print('Data shape after 2-nd pooling: {}'.format(x.shape))
        
        # Для уменьшения размерности тензора используется функция view.
        # Там, где мы хотим зафиксировать размерность, мы ставим -1 (всего может быть только одна -1)
        # А дальше, указываем новые размерности. При этом нужно следить, чтобы произведение всех размерностей у исходного тензора
        # совпадала с произведением размерностей, указанных в этой функции, за исключением -1.
        x = x.view(-1, 16 * 5 * 5)
        # Видим, что после применения функции view тензор из четырехмерного превратился в двухмерный
        # При этом размер второй координаты равен произведению координат 2, 3 и 4 в исходном тензоре
        if self.first_batch:
            # (8, 400)
            print('Data shape after view applying: {}'.format(x.shape))
        
        # Применяем первый полносвязный слой с функцией активации ReLU   
        x = F.relu(self.fc1(x))
        if self.first_batch:
            # (8, 120)
            print('Data shape after 1-st fullyconnected: {}'.format(x.shape))
        
        # Применяем второй полносвязный слой с функцией активации ReLU
        x = F.relu(self.fc2(x))
        if self.first_batch:
            # (8, 84)
            print('Data shape after 2-nd fullyconnected: {}'.format(x.shape))
        
        # Применяем третий полносвязный слой. Важно, чтобы у классификатора на последнем слое не стояло функций активации!
        x = self.fc3(x)
        if self.first_batch:
            # (8, 10)
            print('Data shape after 3-rd fullyconnected: {}'.format(x.shape))
            self.first_batch = False
        return x
    
# Создаем экземпляр нашей сети   
net = Net()
# Переносим сеть на указанное нами устройство
net = net.to(device)

Зададим гиперпараметры обучения нашей сети

In [6]:
N_EPOCHS = 5          # Количество эпох обучения
L_RATE = 0.0001       # Скорость обучения
MOMENTUM = 0.8        # Момент
PRINT_EVERY = 2000    # интервал для вывода результатов

Задаем функцию ошибки и оптимизатор

In [7]:
import torch.optim as optim

# Поскольку у нас задача классификации, где есть лейблы, представленные целыми числами
# Мы используем функцию ошибки перекрестная энтропия
criterion = nn.CrossEntropyLoss()
# В качестве оптимизатора используется стохастический градиентный спуск
# При этом, к скорости обучения мы добавляем еще один параметр - момент,
# который показывает, на сколько быстрее мы должны спускаться по той координате, по которой функция имеет большую скорость убывания
optimizer = optim.SGD(net.parameters(), lr=L_RATE, momentum=MOMENTUM)

Напишем функцию, которая будет вычислять точность нашей сети на тестовых данных. Точность вычисляется по следующей формуле:

$$acc=\frac{количество\_верных\_ответов}{общее\_количество\_ответов}$$

Точность изменяется в диапазоне [0,1]. Часто точность представляют в виде процентов.

In [8]:
def compute_acc(model):
    correct = 0 # Счетчик для общего числа верных ответов
    total = 0 # Счетчик для общего числа ответов
    # Поскольку в ходе тестирования не нужно изменять веса модели, указываем, что градиенты брать не нужно
    with torch.no_grad():
        # Перебираем батчи.
        # testloader - это итератор, который создал батчи за нас.
        # он возвращает пару (входные данные, лейблы)
        for data in testloader:
            images, labels = data
            # Переносим входные данные и лейблы на устройство
            images = images.to(device)
            labels = labels.to(device)
            
            # Посылаем входные данные в сеть и получаем результат работы сети
            outputs = model(images)
            # В качестве предсказанного лейбла выбираем максимальное из 10 значений вектора, получившегося на выходе сети
            # Функция torch.max возвращает два значения: первое - это само максимальное число, а второе - позиция, на которой оно стоит
            # Передаем в функцию тензор и измерение, вдоль которого нужно искать максимум
            _, predicted = torch.max(outputs.data, 1)
            # прибавляем к счетчику общего числа ответов размер батча
            total += labels.size(0)
            # высчитываем количество совпадений результатов работы сети с указанными лейблами
            # и прибовляем это количество к счетчику правильных ответов
            correct += (predicted == labels).sum().item()
    return 100 * correct/ total

Цикл обучения сети. Напоминаю, что каждая итерация перебора всех обучающих примеров называется эпохой. Количество эпох ля нашего случая мы задали выше. В ходе каждой эпохи нужно выполнить несколько действий:
* перебрать все батчи;
* посичтать точность на тестовых данных;
* сохранить веса сети для дальнейшего их использования.

In [9]:
# Перебираем эпохи
for epoch in range(1, N_EPOCHS + 1):
    # Заводим переменную, в которую будем накапливать ошибку после каждого вывода средней ошибки на экран
    running_loss = 0.0
    # Перебираем батчи.
    # trainloader - это итератор, который создал батчи за нас.
    # он возвращает пару (входные данные, лейблы)
    for i, data in enumerate(trainloader):
        inputs, labels = data
        # Переносим входные данные и лейблы на устройство
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        # Обнуляем градиенты
        optimizer.zero_grad()
        
        # Посылаем входные данные в сеть и получаем результат работы сети
        outputs = net(inputs)
        # Считаем функцию ошибки между результатами работы сети и указанными нами лейблами
        loss = criterion(outputs, labels)
        # Считаем градиент
        loss.backward()
        # С помощью оптимизатора делаем шаг в направлении противоположном градиенту
        optimizer.step()
        
        # Прибавляем ошибку на данной итерации к общей ошибке
        # Чтобы превратить тензор, содержащий одно число, в вещественное значение, используют функцию item( )
        running_loss += loss.item()
        # Если номер нашего батча кратен указанному нами числу...
        if i % PRINT_EVERY == 0:
            # выводим на экран среднюю ошибку за указанный промежуток
            print('EPOCH: {}, iter: {} | loss: {:.4f}'.format(epoch, i, running_loss / PRINT_EVERY))
            # и обнуляем накопитель ошибки
            running_loss = 0.0
    
    # Используем написанную нами функцию для подсчета точности на тестовой выборке
    epoch_acc = compute_acc(net)
    print('Accuracy of the network: {:.4}%'.format(epoch_acc))
    
    # Сохраняем веса сети
    torch.save(net.state_dict(), './trained_net_{}_{:.4f}.weight'.format(epoch, epoch_acc))
print('Finished Training')

Input data shape: torch.Size([8, 3, 32, 32])
Data shape after 1-st conv: torch.Size([8, 6, 28, 28])
Data shape after 1-st pooling: torch.Size([8, 6, 14, 14])
Data shape after 2-nd conv: torch.Size([8, 16, 10, 10])
Data shape after 2-nd pooling: torch.Size([8, 16, 5, 5])
Data shape after view applying: torch.Size([8, 400])
Data shape after 1-st fullyconnected: torch.Size([8, 120])
Data shape after 2-nd fullyconnected: torch.Size([8, 84])
Data shape after 3-rd fullyconnected: torch.Size([8, 10])
EPOCH: 1, iter: 0 | loss: 0.0012
EPOCH: 1, iter: 2000 | loss: 2.3021
EPOCH: 1, iter: 4000 | loss: 2.3008
EPOCH: 1, iter: 6000 | loss: 2.2974
Accuracy of the network: 10.97%
EPOCH: 2, iter: 0 | loss: 0.0011
EPOCH: 2, iter: 2000 | loss: 2.2908
EPOCH: 2, iter: 4000 | loss: 2.2822
EPOCH: 2, iter: 6000 | loss: 2.2635
Accuracy of the network: 17.27%
EPOCH: 3, iter: 0 | loss: 0.0011
EPOCH: 3, iter: 2000 | loss: 2.2265
EPOCH: 3, iter: 4000 | loss: 2.1951
EPOCH: 3, iter: 6000 | loss: 2.1453
Accuracy of th

Посмотрим на все файлы с весами, которые были созданы в ходе обучения сети. Последнее вещественное число в названии файла - точность на тестовой выборке для данного набора весов. Соответственно, мы должны использовать в дальнейшей работе файл с самой высокой точностью. Для просмотра содержимого директории можно воспользоваться командой ls. Чтобы выполнять комады операционной системы, а не питона, нужно поставить перед ними восклицательный знак.

In [9]:
!ls


"ls" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


После того, как мы сохранили веса сети, их можно использовать, не обучая сеть каждый раз. Для этого достаточно создать экземпляр сети и загрузить в него веса. Важно помнить, чтобы архитектура экземпляра сети полностью совпадала с архитектурой сети, веса которой были сохранены!

In [13]:
# Создаем новый экземпляр сети
net = Net()
# Считываем веса из файла и загружаем их в сеть
net.load_state_dict(torch.load('./trained_net_5_31.5620.weight'))
# Перносим сеть на устройство, на котором нужно производить вычисления
net = net.to(device)

# В качестве примера выведем точность данного комплекта весов на тестовой выборке.
# Сравним полученную точность с точностью в названии файла и убедимся, что они одинаковы.
acc = compute_acc(net)
print(acc)

Input data shape: torch.Size([8, 3, 32, 32])
Data shape after 1-st conv: torch.Size([8, 6, 28, 28])
Data shape after 1-st pooling: torch.Size([8, 6, 14, 14])
Data shape after 2-nd conv: torch.Size([8, 16, 10, 10])
Data shape after 2-nd pooling: torch.Size([8, 16, 5, 5])
Data shape after view applying: torch.Size([8, 400])
Data shape after 1-st fullyconnected: torch.Size([8, 120])
Data shape after 2-nd fullyconnected: torch.Size([8, 84])
Data shape after 3-rd fullyconnected: torch.Size([8, 10])
31.562


## Упражнение

Давайте применим полученные нами знания на практике и напишем собственный классификатор изображений.

В качестве набора данных будем использовать датасет MNIST. Этот датасет состоит из изображений рукописных цифр от 0 до 9. Каждое изображение имеет размер 28 х 28 пикселей и имеет всего один канал - черно-белый. 

Датасет был представлен Яном Лекуном в 1998 году. Найти более детальную информацию можно на официальном сайте датасета [MNIST](http://yann.lecun.com/exdb/mnist/).

Примеры изображений из датасета MNIST можно увидеть на изображении ниже:

![пример семплов из датасета MNIST](img/mnist.jpeg)

Задайте устройство на котором будут производится вычисления:

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("cuda" if torch.cuda.is_available() else "cpu")

cpu


In [11]:
# Задайте следующие преобразования входных изображений:
# 1) преобразование входного изображения к тензору PyTorch
# 2) нормализация входного изображения со следующими параметрами:
#    математическое ожидание - 0.5, среднеквадратическое отклонение - 0.2 
# Входное изображение черно-белое, по этому у него будет всего один канал
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5), (0.2))])

# Скачивание обучающего множества изображений из набора данных MNIST
trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)

# Создайте генератор батчей для обучающей выборки
# Размер батча задайте равным 64
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                         shuffle=True, num_workers=2)

# Скачивание тестового множества изображений из набора данных MNIST
testset = torchvision.datasets.MNIST(root='./data', train=False,
                                        download=True, transform=transform)
# Создаем генератор батчей для тестовой выборки
# Размер батча задайте равным 64
testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False, num_workers=2)

classes = ('0', '1', '2', '3', '4', '5', '6',
          '7', '8', '9')

Далее зададим архитектуру нашего классификатора. Он будет состоять из следующих слоев:
* сверточный слой, выходных каналов - 4, размер окна - 3, функция активации - ReLU;
* слой Max Pooling, размер окна - 2, сдвиг окна - 2;
* сверточный слой, выходных каналов - 8, размер окна - 3, функция активации - ReLU;
* слой Max Pooling, размер окна - 2, сдвиг окна - 2;
* сверточный слой, выходных каналов - 16, размер окна - 3, функция активации - ReLU;
* полносвязный слой, нейронов в слое - 64, функция активации - ReLU;
* полносвязный слой, нейронов в слое - 32, функция активации - ReLU;
* полносвязный слой, нейронов в слое - 16, функция активации - ReLU;
* полносвязный слой, нейронов в слое - 10.

In [17]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    # Конструктор нашей сети, в котором описываются все слои, которые будут в ней использоваться
    def __init__(self):
        super(Net, self).__init__()
        
        self.pool = nn.MaxPool2d(2, 2)
        self.conv1 = nn.Conv2d(1, 4, 3)
        self.conv2 = nn.Conv2d(4, 8, 3)
        self.conv3 = nn.Conv2d(8, 16, 3)
        
        self.fc1 = nn.Linear(16 * 3 * 3, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(16, 10)
        
    # Опишите последовательность работы слоев сети
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x)) 
        x = x.view(-1, 16 * 3 * 3)
        x = F.relu(self.fc1(x)) 
        x = F.relu(self.fc2(x)) 
        x = F.relu(self.fc3(x)) 
        x = self.fc4(x)
        return x
    
# Создаем экземпляр нашей сети   
net = Net()
# Перенесите сеть на указанное устройство
net = net.to(device)

Зададим гиперпараметры обучения сети:

In [18]:
N_EPOCHS = 10          # Количество эпох обучения
L_RATE = 0.001         # Скорость обучения
MOMENTUM = 0.8         # Момент
PRINT_EVERY = 500      # интервал для вывода результатов

Определим функцию ошибки и оптимизатор. В качестве оптимизатора используйте алгоритм стохастического градиентного спуска.

In [19]:
import torch.optim as optim

# Определим функцию ошибки
criterion = nn.CrossEntropyLoss()
# Создадим оптимизатор
optimizer = optim.SGD(net.parameters(), lr=L_RATE, momentum=MOMENTUM)

Напишите функцию, вычисляющую точность работы сети на тестовой выборке:

In [20]:
def compute_acc(model):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct/ total

Опишите цикл обучения сети. Напоминаю, что каждая эпоха должна состоять из трех этапов:
* перебор всех обучающих батчей
* определение качества работы сети на тестовых данных
* сохранение весов сети

In [21]:
# Перебираем эпохи
for epoch in range(1, N_EPOCHS + 1):
    running_loss = 0.0
    for i, data in enumerate(trainloader):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
        if i % PRINT_EVERY == 0:
            print('EPOCH: {}, iter: {} | loss: {:.4f}'.format(epoch, i, running_loss / PRINT_EVERY))
            running_loss = 0.0
            
    epoch_acc = compute_acc(net)
    print('Accuracy of the network: {:.4}%'.format(epoch_acc))
    
    torch.save(net.state_dict(), './trained_net_{}_{:.4f}.weight'.format(epoch, epoch_acc))
print('Finished Training')

EPOCH: 1, iter: 0 | loss: 0.0046
EPOCH: 1, iter: 500 | loss: 2.3144
Accuracy of the network: 9.82%
EPOCH: 2, iter: 0 | loss: 0.0047
EPOCH: 2, iter: 500 | loss: 2.3006
Accuracy of the network: 13.54%
EPOCH: 3, iter: 0 | loss: 0.0046
EPOCH: 3, iter: 500 | loss: 2.2777
Accuracy of the network: 28.4%
EPOCH: 4, iter: 0 | loss: 0.0043
EPOCH: 4, iter: 500 | loss: 1.9236
Accuracy of the network: 71.63%
EPOCH: 5, iter: 0 | loss: 0.0018
EPOCH: 5, iter: 500 | loss: 0.7390
Accuracy of the network: 82.13%
EPOCH: 6, iter: 0 | loss: 0.0011
EPOCH: 6, iter: 500 | loss: 0.5147
Accuracy of the network: 86.02%
EPOCH: 7, iter: 0 | loss: 0.0007
EPOCH: 7, iter: 500 | loss: 0.4157
Accuracy of the network: 88.63%
EPOCH: 8, iter: 0 | loss: 0.0007
EPOCH: 8, iter: 500 | loss: 0.3507
Accuracy of the network: 91.21%
EPOCH: 9, iter: 0 | loss: 0.0005
EPOCH: 9, iter: 500 | loss: 0.2947
Accuracy of the network: 92.58%
EPOCH: 10, iter: 0 | loss: 0.0004
EPOCH: 10, iter: 500 | loss: 0.2484
Accuracy of the network: 92.58%


Создайте новый экземпляр класса сети, и загрузите в него обученные веса с самой высокой точностью на тестовых данных. Посчитайте точность для новой сети. Сравните ее с максимальной точностью, полученной в ходе обучения сети.

In [22]:
net = Net()
net.load_state_dict(torch.load('./trained_net_10_92.5800.weight'))
net = net.to(device)
acc = compute_acc(net)
print(acc)

92.58
